# Imports

In [ ]:
import pandas as pd

from ipynb.fs.full.utility_functions import (
    get_price_data,
    get_strategy_stats,
)
from datetime import datetime
from enum import Enum

# Price Data Collection

In [ ]:
univ = [
    "BTCUSDT", "ETHUSDT", "ADAUSDT", "BNBUSDT", "XRPUSDT", "DOTUSDT", "MATICUSDT", "LTCUSDT", "BCHUSDT",
    "LINKUSDT", "XLMUSDT", "USDCUSDT", "EOSUSDT", "TRXUSDT", "XTZUSDT", "FILUSDT", "NEOUSDT", "DAIUSDT",
    "DASHUSDT", "VETUSDT", "ATOMUSDT", "AAVEUSDT", "UNIUSDT", "GRTUSDT", "THETAUSDT", "IOTAUSDT", "BUSDUSDT",
    "ZECUSDT", "YFIUSDT", "ETCUSDT", "WAVESUSDT", "COMPUSDT", "SNXUSDT", "DOGEUSDT", "MKRUSDT", "ZILUSDT",
    "SUSHIUSDT", "KSMUSDT", "OMGUSDT", "ONTUSDT", "ALGOUSDT", "EGLDUSDT", "BATUSDT", "DGBUSDT", "ZRXUSDT",
    "TUSDUSDT", "QTUMUSDT", "ICXUSDT", "AVAXUSDT", "RENUSDT", "HBARUSDT", "NEARUSDT", "LRCUSDT", "CELOUSDT",
    "KNCUSDT", "LSKUSDT", "OCEANUSDT", "QNTUSDT", "USTUSDT", "BANDUSDT", "MANAUSDT", "ENJUSDT", "ANTUSDT",
    "BNTUSDT", "ZENUSDT", "NMRUSDT", "RVNUSDT", "IOSTUSDT", "OXTUSDT", "CRVUSDT", "MATICUSDT", "HNTUSDT",
    "BALUSDT", "CHZUSDT"
]

px_4h = get_price_data(univ, '4h', True, './class_project_input_prices.csv')
px_4h

In [8]:
class CointIdMethod(Enum):
    FIXED_TIME = 1
    EVERY_SIX_MONTHS = 2
    EVERY_THREE_MONTHS = 3


freq_to_px = {
    4: px_4h,
    8: px_4h.loc[pd.date_range(px_4h.index[0], px_4h.index[-1], freq='8h').drop(datetime(2023, 2, 6, 8, 0))],
    12: px_4h.loc[pd.date_range(px_4h.index[0], px_4h.index[-1], freq='12h')],
    24: px_4h.loc[pd.date_range(px_4h.index[0], px_4h.index[-1], freq='24h')],
    48: px_4h.loc[pd.date_range(px_4h.index[0], px_4h.index[-1], freq='48h')],
}


def get_fixed_time_to_coint_pairs(px_coint, px_trade_sim, significance_level):
    pass


def get_six_months_time_to_coint_pairs(px_full_sim, px_trade_sim, significance_level):
    pass


def get_three_months_time_to_coint_pairs(px_full_sim, px_trade_sim, significance_level):
    pass


# return map from times in px_full_sim to a set of cointegrated pairs at that time and price dataframe to
# trade during
def get_cointegrated_pairs(px_full_sim, cointegration_identification_method, significance_level):
    trade_period_start = datetime(2022, 1, 1, 0, 0)
    px_trade_sim = px_full_sim.loc[trade_period_start:]
    
    if cointegration_identification_method == CointIdMethod.FIXED_TIME:
        px_coint = px_full_sim.loc[:trade_period_start].iloc[:-1]
        time_to_coint_pairs = get_fixed_time_to_coint_pairs(px_coint, px_trade_sim, significance_level)
        return time_to_coint_pairs, px_trade_sim
    elif cointegration_identification_method == CointIdMethod.EVERY_SIX_MONTHS:
        time_to_coint_pairs = get_six_months_time_to_coint_pairs(px_full_sim, px_trade_sim, significance_level)
        return time_to_coint_pairs, px_trade_sim
    elif cointegration_identification_method == CointIdMethod.EVERY_THREE_MONTHS:
        time_to_coint_pairs = get_three_months_time_to_coint_pairs(
            px_full_sim,
            px_trade_sim,
            significance_level
        )
        return time_to_coint_pairs, px_trade_sim
    else:
        time_to_coint_pairs = dict()
        for time_index in px_trade_sim.index:
            time_to_coint_pairs[time_index] = set()
        return time_to_coint_pairs, px_trade_sim


def has_coint_pair_reached_exit_threshold(
    coint_coin_a,
    coint_coin_b,
    hold_status,
    px_trade_sim,
    open_time,
    beta_calculation_window,
    z_score_exit_level
):
    pass



# return gross returns and net returns
def run_trade_simulation(
    px_trade_sim,
    trade_hours_freq,
    time_to_coint_pairs,
    beta_calculation_window,
    z_score_entry_level,
    z_score_exit_level
):
    share_positions = pd.DataFrame(0.0, columns = px_trade_sim.columns, index = px_trade_sim.index)
    pairs_currently_held = dict()
    
    for index in len(px_trade_sim):
        open_time = px_trade_sim.index[index]
        pairs_to_remove_from_holdings = set()
        
        if index > 0:
            share_positions.iloc[index] = share_positions.iloc[index-1]
        
        for (coint_coin_a, coint_coin_b, hold_status), (share_size_a, share_size_b) in pairs_currently_held.items():
            
            if has_coint_pair_reached_exit_threshold(
                coint_coin_a,
                coint_coin_b,
                hold_status,
                px_trade_sim,
                open_time,
                beta_calculation_window,
                z_score_exit_level
            ):
                share_positions.loc[open_time, coint_coin_a] -= share_size_a
                share_positions.loc[open_time, coint_coin_b] -= share_size_b
                pairs_to_remove_from_holdings.add((coint_coin_a, coint_coin_b, hold_status))
        
        for entry in pairs_to_remove_from_holdings:
            del pairs_currently_held[entry]
        
        
        for (coint_coin_a, coint_coin_b) in time_to_coint_pairs[open_time]:
            
            hold_status, beta = has_coint_pair_reached_entry_threshold(
                coint_coin_a,
                coint_coin_b,
                px_trade_sim,
                open_time,
                beta_calculation_window,
                z_score_entry_level
            )
            
            if hold_status == "BUY": # z_score(coint_coin_a - beta * coint_coin_b) < -entry_level
                raw_share_size_a = 1
                raw_share_size_b = -beta
                normalization_factor = abs(raw_share_size_a) + abs(raw_share_size_b)
                share_size_a = raw_share_size_a / normalization_factor
                share_size_b = raw_share_size_b / normalization_factor
                
                pairs_currently_held[(coint_coin_a, coint_coin_b, hold_status)] = (share_size_a, share_size_b)
                share_positions.loc[open_time, coint_coin_a] += share_size_a
                share_positions.loc[open_time, coint_coin_b] += share_size_b
                
            elif hold_status == "SELL": # z_score(coint_coin_a - beta * coint_coin_b) > entry_level
                raw_share_size_a = -1
                raw_share_size_b = beta
                normalization_factor = abs(raw_share_size_a) + abs(raw_share_size_b)
                share_size_a = raw_share_size_a / normalization_factor
                share_size_b = raw_share_size_b / normalization_factor
                
                pairs_currently_held[(coint_coin_a, coint_coin_b, hold_status)] = (share_size_a, share_size_b)
                share_positions.loc[open_time, coint_coin_a] += share_size_a
                share_positions.loc[open_time, coint_coin_b] += share_size_b
    
    gross_positions = px_trade_sim * share_positions








def get_string_enum(cointegration_identification_method):
    if cointegration_identification_method == CointIdMethod.FIXED_TIME:
        return "FIXED_TIME"
    elif cointegration_identification_method == CointIdMethod.EVERY_SIX_MONTHS:
        return "EVERY_SIX_MONTHS"
    elif cointegration_identification_method == CointIdMethod.EVERY_THREE_MONTHS:
        return "EVERY_THREE_MONTHS"
    else:
        return ""


# construct file names for gross returns and net returns
def construct_file_names(
    trade_hours_freq,
    cointegration_identification_method,
    significance_level,
    beta_calculation_window,
    z_score_entry_level,
    z_score_exit_level
):
    t = str(trade_hours_freq)
    c = get_string_enum(cointegration_identification_method)
    s = str(significance_level)
    b = str(beta_calculation_window)
    n = str(z_score_entry_level)
    x = str(z_score_exit_level)
    return (
        f"simulation_data/gross_rets_{t}_{c}_{s}_{b}_{n}_{x}",
        f"simulation_data/net_rets_{t}_{c}_{s}_{b}_{n}_{x}"
    )


def construct_variant_key(
    trade_hours_freq,
    cointegration_identification_method,
    significance_level,
    beta_calculation_window,
    z_score_entry_level,
    z_score_exit_level
):
    return (
        trade_hours_freq,
        get_string_enum(cointegration_identification_method),
        significance_level,
        beta_calculation_window,
        z_score_entry_level,
        z_score_exit_level
    )


results = dict()

for trade_hours_freq in (4, 8, 12, 24, 48):
    px_full_sim = freq_to_px[trade_hours_freq]

    for cointegration_identification_method in (
        CointIdMethod.FIXED_TIME, CointIdMethod.EVERY_SIX_MONTHS, CointIdMethod.EVERY_THREE_MONTHS
    ):
        for significance_level in (1e-8, 1e-6, 1e-4, 0.01, 0.02, 0.05):
            time_to_coint_pairs, px_trade_sim = get_cointegrated_pairs(
                px_full_sim,
                cointegration_identification_method,
                significance_level
            )
            
            for beta_calculation_window in (
                int(24 / trade_hours_freq * 7),
                int(24 / trade_hours_freq * 7 * 2),
                int(24 / trade_hours_freq * 7 * 5),
                int(24 / trade_hours_freq * 7 * 10),
                int(24 / trade_hours_freq * 7 * 20),
                int(24 / trade_hours_freq * 7 * 40),
                int(24 / trade_hours_freq * 7 * 80),
            ):
                for z_score_entry_level in (1.0, 1.5, 1.75, 2.0, 2.5, 3.0):
                    for z_score_exit_level in (
                        z_score_entry_level / 2,
                        z_score_entry_level - 1,
                        (z_score_entry_level - 1) / 2
                    ):
                        gross_returns, net_returns = run_trade_simulation(
                            px_trade_sim,
                            trade_hours_freq,
                            time_to_coint_pairs,
                            beta_calculation_window,
                            z_score_entry_level,
                            z_score_exit_level
                        )
                        
                        gross_ret_file_name, net_ret_file_name = construct_file_names(
                            trade_hours_freq,
                            cointegration_identification_method,
                            significance_level,
                            beta_calculation_window,
                            z_score_entry_level,
                            z_score_exit_level
                        )
                        
                        gross_returns.to_csv(gross_ret_file_name)
                        net_returns.to_csv(net_ret_file_name)

                        variant_key = construct_variant_key(
                            trade_hours_freq,
                            cointegration_identification_method,
                            significance_level,
                            beta_calculation_window,
                            z_score_entry_level,
                            z_score_exit_level
                        )
                        
                        results[variant_key] = get_strategy_stats(net_returns, trade_hours_freq, px_trade_sim)

results_df = pd.DataFrame(results)

In [5]:
df1 = pd.DataFrame({
    'X': [1, 2, 3, 4, 5, 6],
    'Y': [2, 5, 7, 8, 9, 0],
})

df1

,X,Y
0,1,2
1,2,5
2,3,7
3,4,8
4,5,9
5,6,0


In [6]:
df2 = pd.DataFrame({
    'X': [1, 2, 1, 1, 1, 2],
    'Y': [2, 3, 1, 2, 3, 1],
})

df2

,X,Y
0,1,2
1,2,3
2,1,1
3,1,2
4,1,3
5,2,1


In [7]:
df1 * df2

,X,Y
0,1,4
1,4,15
2,3,7
3,4,16
4,5,27
5,12,0
